# Importing Libraries

In [149]:
import random
import json
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from tensorflow.keras.layers import Embedding, LSTM, Dense,SimpleRNN,Dropout,Bidirectional

# Loading the Simulated data

In [178]:
# Load the simulated data
with open('simulated_purchase_history.json', 'r') as file:
    simulated_data = json.load(file)
simulated_data

[{'user': 'user1', 'purchases': ['amazon', 'myntra', 'flipkart']},
 {'user': 'user2', 'purchases': ['amazon']},
 {'user': 'user3', 'purchases': ['myntra']},
 {'user': 'user4', 'purchases': ['myntra']},
 {'user': 'user5', 'purchases': ['myntra', 'amazon', 'flipkart']},
 {'user': 'user6', 'purchases': ['amazon']},
 {'user': 'user7', 'purchases': ['amazon']},
 {'user': 'user8', 'purchases': ['myntra']},
 {'user': 'user9', 'purchases': ['amazon', 'myntra', 'flipkart']},
 {'user': 'user10', 'purchases': ['flipkart']},
 {'user': 'user11', 'purchases': ['flipkart', 'amazon', 'myntra']},
 {'user': 'user12', 'purchases': ['flipkart', 'myntra', 'amazon']},
 {'user': 'user13', 'purchases': ['amazon']},
 {'user': 'user14', 'purchases': ['amazon', 'myntra']},
 {'user': 'user15', 'purchases': ['amazon', 'flipkart']},
 {'user': 'user16', 'purchases': ['myntra', 'flipkart']},
 {'user': 'user17', 'purchases': ['myntra']},
 {'user': 'user18', 'purchases': ['myntra', 'amazon']},
 {'user': 'user19', 'purc

# Preprocessing the data

In [153]:
# Extract purchase sequences
purchase_sequences = [entry['purchases'] for entry in simulated_data]


# Tokenization and encoding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(purchase_sequences)
encoded_sequences = tokenizer.texts_to_sequences(purchase_sequences)


# Preparing data for RNN
max_sequence_len = max(len(seq) for seq in encoded_sequences)
padded_sequences = pad_sequences(encoded_sequences, maxlen=max_sequence_len, padding='pre')

# Data Splitting

In [155]:
# Preparing predictors and label sequences
X = padded_sequences[:, :-1]
y = padded_sequences[:, -1]

# Spilliting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# RNN Model

In [179]:
embedding_size = 20
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_size, input_length=max_sequence_len - 1))
model.add(SimpleRNN(100, return_sequences=True))
model.add(Dropout(0.3))
model.add(SimpleRNN(100))
model.add(Dropout(0.3))
model.add(Dense(len(tokenizer.word_index) + 1, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/100
22/22 [==============================] - 9s 76ms/step - loss: 1.2156 - accuracy: 0.4214 - val_loss: 1.0400 - val_accuracy: 0.5500
Epoch 2/100
22/22 [==============================] - 0s 15ms/step - loss: 0.9754 - accuracy: 0.5486 - val_loss: 0.8415 - val_accuracy: 0.5933
Epoch 3/100
22/22 [==============================] - 0s 16ms/step - loss: 0.7922 - accuracy: 0.5943 - val_loss: 0.6766 - val_accuracy: 0.6633
Epoch 4/100
22/22 [==============================] - 0s 16ms/step - loss: 0.7049 - accuracy: 0.6200 - val_loss: 0.6679 - val_accuracy: 0.6267
Epoch 5/100
22/22 [==============================] - 0s 16ms/step - loss: 0.6450 - accuracy: 0.6257 - val_loss: 0.6311 - val_accuracy: 0.6267
Epoch 6/100
22/22 [==============================] - 0s 16ms/step - loss: 0.6434 - accuracy: 0.6100 - val_loss: 0.6198 - val_accuracy: 0.6267
Epoch 7/100
22/22 [==============================] - 0s 16ms/step - loss: 0.6407 - accuracy: 0.6043 - val_loss: 0.6164 - val_accuracy: 0.6167
Epoch 

22/22 [==============================] - 0s 14ms/step - loss: 0.6153 - accuracy: 0.6214 - val_loss: 0.6054 - val_accuracy: 0.6267
Epoch 59/100
22/22 [==============================] - 0s 14ms/step - loss: 0.6154 - accuracy: 0.6257 - val_loss: 0.5895 - val_accuracy: 0.6300
Epoch 60/100
22/22 [==============================] - 0s 14ms/step - loss: 0.6184 - accuracy: 0.6057 - val_loss: 0.5936 - val_accuracy: 0.6267
Epoch 61/100
22/22 [==============================] - 0s 16ms/step - loss: 0.6101 - accuracy: 0.6300 - val_loss: 0.5894 - val_accuracy: 0.6200
Epoch 62/100
22/22 [==============================] - 0s 16ms/step - loss: 0.6138 - accuracy: 0.5943 - val_loss: 0.5917 - val_accuracy: 0.6167
Epoch 63/100
22/22 [==============================] - 0s 14ms/step - loss: 0.6162 - accuracy: 0.6200 - val_loss: 0.5972 - val_accuracy: 0.6267
Epoch 64/100
22/22 [==============================] - 0s 15ms/step - loss: 0.6165 - accuracy: 0.6071 - val_loss: 0.6049 - val_accuracy: 0.6267
Epoch 65/100

# Model Evaluation

In [182]:
# Accuracy on test data

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

10/10 [==============================] - 0s 6ms/step - loss: 0.5926 - accuracy: 0.6300
Test Accuracy: 0.6300


# Classification Report

In [184]:
# Predictions
probabilities = model.predict(X_test)
predictions = np.argmax(probabilities, axis=1)

# Classification report
print(classification_report(y_test, predictions, target_names=tokenizer.word_index.keys()))



10/10 [==============================] - 0s 5ms/step
              precision    recall  f1-score   support

    flipkart       0.70      0.72      0.71       103
      myntra       1.00      0.33      0.50        99
      amazon       0.51      0.84      0.63        98

    accuracy                           0.63       300
   macro avg       0.74      0.63      0.61       300
weighted avg       0.74      0.63      0.62       300



# Challenges faced during the Development :

1] Data was not provided so simulated data may not fully capture the complexities  of real user purchase histories.Generating a  dataset with a balanced distribution of user purchase histories across different platforms and product categories was challenging. 

2] Choosing the appropriate deep learning model (RNN, LSTM) for sequence prediction was challenging.Got the better accuracy of RNN model so I used that model . 

3] Deep learning models based on recurrent neural networks, are computationally expensive to train, requiring substantial computational resources and time.

4] To ensure model is not overfitting and generalizes well to unseen data , techniques such as regularization, dropout, and early sto were used.



These were the challenges faced and how I overcome those challenges.



